In [12]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.2f}'.format

In [13]:
cotacao_trim = pd.read_csv(r"C:\Users\usuario\Desktop\Programacao\financeml\src\data\dados_tratados\cotacao_trimestral.csv")

In [14]:
# Agrupar pelo código de negociação
cotacao_agrupado = cotacao_trim.groupby('cod_negociacao').apply(lambda x: x.sort_values('data_pregao')).reset_index(drop=True)

# Mostrar o resultado
cotacao_agrupado

C:\Users\usuario\AppData\Local\Temp\ipykernel_9664\1169394240.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cotacao_agrupado = cotacao_trim.groupby('cod_negociacao').apply(lambda x: x.sort_values('data_pregao')).reset_index(drop=True)


,data_pregao,CNPJ,cod_negociacao,noma_empresa,preco_ultimo_negocio,noma_empresa_padronizado
0,2016-12-29,42771949000135.00,AALR3,ALLIAR,14.65,alliar
1,2017-03-31,42771949000135.00,AALR3,ALLIAR,15.63,alliar
2,2017-05-31,42771949000135.00,AALR3,ALLIAR,17.73,alliar
3,2017-06-30,42771949000135.00,AALR3,ALLIAR,17.54,alliar
4,2017-08-31,42771949000135.00,AALR3,ALLIAR,16.40,alliar
...,...,...,...,...,...,...
12587,2023-06-30,13574594000196.00,ZAMP3,ZAMP S.A.,4.55,zamp
12588,2023-08-31,13574594000196.00,ZAMP3,ZAMP S.A.,5.55,zamp
12589,2023-09-29,13574594000196.00,ZAMP3,ZAMP S.A.,6.13,zamp
12590,2023-11-30,13574594000196.00,ZAMP3,ZAMP S.A.,5.82,zamp


In [15]:
# Ordenar os dados por código de negociação e data do pregão
cotacao_trim = cotacao_trim.sort_values(by=['cod_negociacao', 'data_pregao'])

# Calcular a variação percentual trimestral
cotacao_trim['variacao_percentual'] = cotacao_trim.groupby('cod_negociacao')['preco_ultimo_negocio'].pct_change() * 100

# Reordenar as colunas para que 'variacao_percentual' esteja ao lado de 'preco_ultimo_negocio'
cols = cotacao_trim.columns.tolist()
cols.insert(cols.index('preco_ultimo_negocio') + 1, cols.pop(cols.index('variacao_percentual')))
cotacao_trim = cotacao_trim[cols]

# Mostrar o resultado
cotacao_trim


,data_pregao,CNPJ,cod_negociacao,noma_empresa,preco_ultimo_negocio,variacao_percentual,noma_empresa_padronizado
1646,2016-12-29,42771949000135.00,AALR3,ALLIAR,14.65,NaN,alliar
1786,2017-03-31,42771949000135.00,AALR3,ALLIAR,15.63,6.69,alliar
1934,2017-05-31,42771949000135.00,AALR3,ALLIAR,17.73,13.44,alliar
2089,2017-06-30,42771949000135.00,AALR3,ALLIAR,17.54,-1.07,alliar
2241,2017-08-31,42771949000135.00,AALR3,ALLIAR,16.40,-6.50,alliar
...,...,...,...,...,...,...,...
11307,2023-06-30,13574594000196.00,ZAMP3,ZAMP S.A.,4.55,19.42,zamp
11594,2023-08-31,13574594000196.00,ZAMP3,ZAMP S.A.,5.55,21.98,zamp
11898,2023-09-29,13574594000196.00,ZAMP3,ZAMP S.A.,6.13,10.45,zamp
12081,2023-11-30,13574594000196.00,ZAMP3,ZAMP S.A.,5.82,-5.06,zamp


In [17]:
# Ensure 'data_pregao' is in datetime format
cotacao_trim["data_pregao"] = pd.to_datetime(cotacao_trim["data_pregao"])

# Now you can use the .dt accessor
cotacao_trim["data_pregao_mes_ano"] = cotacao_trim["data_pregao"].dt.to_period('M')
cotacao_trim_resume = cotacao_trim[["cod_negociacao", "data_pregao_mes_ano", "preco_ultimo_negocio", "variacao_percentual"]]

# Display the result
cotacao_trim_resume

,cod_negociacao,data_pregao_mes_ano,preco_ultimo_negocio,variacao_percentual
1646,AALR3,2016-12,14.65,NaN
1786,AALR3,2017-03,15.63,6.69
1934,AALR3,2017-05,17.73,13.44
2089,AALR3,2017-06,17.54,-1.07
2241,AALR3,2017-08,16.40,-6.50
...,...,...,...,...
11307,ZAMP3,2023-06,4.55,19.42
11594,ZAMP3,2023-08,5.55,21.98
11898,ZAMP3,2023-09,6.13,10.45
12081,ZAMP3,2023-11,5.82,-5.06


In [ ]:
dre_trim = pd.read_csv(r"C:\Users\usuario\Desktop\Programacao\financeml\src\data\dados_tratados\dre_tri.csv")

In [20]:
dre_trim

,Unnamed: 0,CNPJ_CIA,DENOM_CIA,CD_CVM,GRUPO_DFP,DT_FIM_EXERC,Corrente,Custo dos Bens e/ou Serviços Vendidos,Despesas Financeiras,Despesas Gerais e Administrativas,...,Resultado Financeiro,Resultado Líquido das Operações Continuadas,Resultado Líquido de Operações Descontinuadas,Resultado de Equivalência Patrimonial,Margem Bruta,Margem Operacional,Margem Líquida,Cobertura de Juros,Turnover de Ativos,data_pregao_mes_ano
0,60,00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Individual - Demonstração do Resultado,2014-03-31,-41937.00,-27198.00,-506268.00,-1580488.00,...,372987.00,985903.00,0.00,1524978.00,0.96,0.89,1.34,-2.03,-17.59,2014-03
1,61,00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Individual - Demonstração do Resultado,2014-06-30,41937.00,-155592.00,-1412804.00,-5492590.00,...,1295191.00,776655.00,0.00,2716684.00,0.93,-0.24,0.33,-0.52,56.54,2014-06
2,62,00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Individual - Demonstração do Resultado,2014-09-30,0.00,-3048196.00,-1441332.00,-6405363.00,...,2255191.00,-4595661.00,0.00,-329161.00,-0.04,-2.34,-1.57,3.19,inf,2014-09
3,63,00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Individual - Demonstração do Resultado,2015-03-31,0.00,-693124.00,-644404.00,-886395.00,...,1242593.00,1255247.00,0.00,971927.00,0.14,0.24,1.56,-2.23,inf,2015-03
4,64,00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Individual - Demonstração do Resultado,2015-06-30,-84307.00,-2096638.00,-9523312.00,-3956953.00,...,2114309.00,-1460973.00,0.00,277431.00,0.14,-1.37,-0.60,0.13,-28.86,2015-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18419,19672,97.837.181/0001-47,DURATEX S.A.,21091,DF Individual - Demonstração do Resultado,2015-06-30,0.00,-1990642.00,-212038.00,-98038.00,...,-157125.00,140429.00,0.00,119176.00,0.23,0.10,0.05,-0.43,inf,2015-06
18420,19673,97.837.181/0001-47,DURATEX S.A.,21091,DF Individual - Demonstração do Resultado,2015-09-30,0.00,-2718385.00,-335064.00,-135513.00,...,-187653.00,159853.00,0.00,141426.00,0.24,0.09,0.04,-0.39,inf,2015-09
18421,19674,97.837.181/0001-47,DURATEX S.A.,21091,DF Individual - Demonstração do Resultado,2016-03-31,-2181.00,-643380.00,-73832.00,-34302.00,...,-55973.00,-31122.00,0.00,31842.00,0.15,-0.01,-0.04,0.85,-346.08,2016-03
18422,19675,97.837.181/0001-47,DURATEX S.A.,21091,DF Individual - Demonstração do Resultado,2016-06-30,-59005.00,-1936462.00,-226024.00,-92242.00,...,-175063.00,-31356.00,0.00,114904.00,0.18,0.03,-0.01,0.46,-39.97,2016-06


In [21]:
dre_trim['DT_FIM_EXERC'] = pd.to_datetime(dre_trim['DT_FIM_EXERC'])
dre_trim["data_pregao_mes_ano"] = dre_trim["DT_FIM_EXERC"].dt.to_period('M')
dre_trim
                                        
                                          
                                          

,Unnamed: 0,CNPJ_CIA,DENOM_CIA,CD_CVM,GRUPO_DFP,DT_FIM_EXERC,Corrente,Custo dos Bens e/ou Serviços Vendidos,Despesas Financeiras,Despesas Gerais e Administrativas,...,Resultado Financeiro,Resultado Líquido das Operações Continuadas,Resultado Líquido de Operações Descontinuadas,Resultado de Equivalência Patrimonial,Margem Bruta,Margem Operacional,Margem Líquida,Cobertura de Juros,Turnover de Ativos,data_pregao_mes_ano
0,60,00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Individual - Demonstração do Resultado,2014-03-31,-41937.00,-27198.00,-506268.00,-1580488.00,...,372987.00,985903.00,0.00,1524978.00,0.96,0.89,1.34,-2.03,-17.59,2014-03
1,61,00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Individual - Demonstração do Resultado,2014-06-30,41937.00,-155592.00,-1412804.00,-5492590.00,...,1295191.00,776655.00,0.00,2716684.00,0.93,-0.24,0.33,-0.52,56.54,2014-06
2,62,00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Individual - Demonstração do Resultado,2014-09-30,0.00,-3048196.00,-1441332.00,-6405363.00,...,2255191.00,-4595661.00,0.00,-329161.00,-0.04,-2.34,-1.57,3.19,inf,2014-09
3,63,00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Individual - Demonstração do Resultado,2015-03-31,0.00,-693124.00,-644404.00,-886395.00,...,1242593.00,1255247.00,0.00,971927.00,0.14,0.24,1.56,-2.23,inf,2015-03
4,64,00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Individual - Demonstração do Resultado,2015-06-30,-84307.00,-2096638.00,-9523312.00,-3956953.00,...,2114309.00,-1460973.00,0.00,277431.00,0.14,-1.37,-0.60,0.13,-28.86,2015-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18419,19672,97.837.181/0001-47,DURATEX S.A.,21091,DF Individual - Demonstração do Resultado,2015-06-30,0.00,-1990642.00,-212038.00,-98038.00,...,-157125.00,140429.00,0.00,119176.00,0.23,0.10,0.05,-0.43,inf,2015-06
18420,19673,97.837.181/0001-47,DURATEX S.A.,21091,DF Individual - Demonstração do Resultado,2015-09-30,0.00,-2718385.00,-335064.00,-135513.00,...,-187653.00,159853.00,0.00,141426.00,0.24,0.09,0.04,-0.39,inf,2015-09
18421,19674,97.837.181/0001-47,DURATEX S.A.,21091,DF Individual - Demonstração do Resultado,2016-03-31,-2181.00,-643380.00,-73832.00,-34302.00,...,-55973.00,-31122.00,0.00,31842.00,0.15,-0.01,-0.04,0.85,-346.08,2016-03
18422,19675,97.837.181/0001-47,DURATEX S.A.,21091,DF Individual - Demonstração do Resultado,2016-06-30,-59005.00,-1936462.00,-226024.00,-92242.00,...,-175063.00,-31356.00,0.00,114904.00,0.18,0.03,-0.01,0.46,-39.97,2016-06


In [22]:
dre_trim.columns

Index(['Unnamed: 0', 'CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'GRUPO_DFP',
       'DT_FIM_EXERC', 'Corrente', 'Custo dos Bens e/ou Serviços Vendidos',
       'Despesas Financeiras', 'Despesas Gerais e Administrativas',
       'Despesas/Receitas Operacionais', 'Diferido',
       'Imposto de Renda e Contribuição Social sobre o Lucro',
       'Lucro/Prejuízo Líquido das Operações Descontinuadas',
       'Lucro/Prejuízo do Período', 'Outras Despesas Operacionais',
       'Perdas pela Não Recuperabilidade de Ativos',
       'Receita de Venda de Bens e/ou Serviços', 'Receitas Financeiras',
       'Resultado Antes do Resultado Financeiro e dos Tributos',
       'Resultado Antes dos Tributos sobre o Lucro', 'Resultado Bruto',
       'Resultado Financeiro', 'Resultado Líquido das Operações Continuadas',
       'Resultado Líquido de Operações Descontinuadas',
       'Resultado de Equivalência Patrimonial', 'Margem Bruta',
       'Margem Operacional', 'Margem Líquida', 'Cobertura de Juros',
       'Tur

In [23]:
# Identificar os códigos de negociação comuns em ambas as planilhas
common_cod_negociacao = set(cotacao_trim['cod_negociacao']).intersection(set(dre_trim['cod_negociacao']))

# Filtrar cotacao_trim para manter apenas as empresas comuns
cotacao_trim_filtered = cotacao_trim[cotacao_trim['cod_negociacao'].isin(common_cod_negociacao)]

# Filtrar balance_trim para manter apenas as empresas comuns
balance_trim_filtered = dre_trim[dre_trim['cod_negociacao'].isin(common_cod_negociacao)]

# Mostrar a quantidade de empresas em cada planilha após o filtro
print(f"Empresas comuns: {len(common_cod_negociacao)}")
print(f"Linhas em cotacao_trim_filtered: {len(cotacao_trim_filtered)}")
print(f"Linhas em balance_trim_filtered: {len(balance_trim_filtered)}")

KeyError: 'cod_negociacao'

In [ ]:
dre_trim_resume = cotacao_trim[["cod_negociacao","data_pregao_mes_ano","preco_ultimo_negocio","variacao_percentual"]]

cotacao_trim_resume


Index(['Unnamed: 0', 'CNPJ_CIA', 'DENOM_CIA', 'CD_CVM', 'GRUPO_DFP',
       'DT_FIM_EXERC', 'Corrente', 'Custo dos Bens e/ou Serviços Vendidos',
       'Despesas Financeiras', 'Despesas Gerais e Administrativas',
       'Despesas/Receitas Operacionais', 'Diferido',
       'Imposto de Renda e Contribuição Social sobre o Lucro',
       'Lucro/Prejuízo Líquido das Operações Descontinuadas',
       'Lucro/Prejuízo do Período', 'Outras Despesas Operacionais',
       'Perdas pela Não Recuperabilidade de Ativos',
       'Receita de Venda de Bens e/ou Serviços', 'Receitas Financeiras',
       'Resultado Antes do Resultado Financeiro e dos Tributos',
       'Resultado Antes dos Tributos sobre o Lucro', 'Resultado Bruto',
       'Resultado Financeiro', 'Resultado Líquido das Operações Continuadas',
       'Resultado Líquido de Operações Descontinuadas',
       'Resultado de Equivalência Patrimonial', 'Margem Bruta',
       'Margem Operacional', 'Margem Líquida', 'Cobertura de Juros',
       'Tur